In [7]:
import torch
import torch.nn as nn
from trl import RLOOConfig, RLOOTrainer, apply_chat_template
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

secret_value_0 = os.getenv('HF_TOKEN')

from huggingface_hub import login
login(token=secret_value_0)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

# Create a custom reward model that always returns a constant reward
class ConstantRewardModel(nn.Module):
    def __init__(self):
        super(ConstantRewardModel, self).__init__()

    def forward(self, outputs):
        # Return a tensor of constant rewards (1.0)
        return torch.ones(outputs.size(0), 1)  # Shape: (batch_size, 1)

# Instantiate the custom reward model
reward_model = ConstantRewardModel()

# Add necessary attributes to the reward model
reward_model.config = {"num_labels": 1}  # Mimic the config attribute
reward_model.base_model_prefix = "constant_reward_model"

ref_policy = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
policy = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

dataset = load_dataset("trl-lib/ultrafeedback-prompt")
dataset = dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})
dataset = dataset.map(lambda x: tokenizer(x["prompt"]), remove_columns="prompt")

training_args = RLOOConfig(output_dir="Qwen2.5-0.5B-RL", report_to="none")
trainer = RLOOTrainer(
    config=training_args,
    processing_class=tokenizer,
    policy=policy,
    ref_policy=ref_policy,
    reward_model=reward_model,  # Use the custom reward model here
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)
trainer.train()


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


===training policy===


AttributeError: 'ConstantRewardModel' object has no attribute 'constant_reward_model'

In [1]:
from trl import RLOOConfig, RLOOTrainer, apply_chat_template, PPOConfig, PPOTrainer
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
import torch
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

secret_value_0 = os.getenv('HF_TOKEN')

from huggingface_hub import login
login(token = secret_value_0)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
reward_model = AutoModelForSequenceClassification.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct", num_labels=1
)

def custom_reward(output_tensors):
    results = []
    for output in output_tensors:
        txt = tokenizer.decode(output)
        print(txt)
        results.append(1)
    
    return torch.tensor(results)

ref_policy = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
policy = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

dataset = load_dataset("trl-lib/ultrafeedback-prompt")
dataset = dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})
dataset = dataset.map(lambda x: tokenizer(x["prompt"]), remove_columns="prompt")

training_args = RLOOConfig(output_dir="Qwen2.5-0.5B-RL", report_to="none")
trainer = RLOOTrainer(
    config=training_args,
    processing_class=tokenizer,
    policy=policy,
    ref_policy=ref_policy,
    reward_model=reward_model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)
trainer.train()

/home/jj/miniforge3/envs/automode_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


===training policy===


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ system                                      │  Cristiano Ronaldo. He is the current       │ -2.372755765914917  │
│ You are Qwen, created by Alibaba Cloud. You │ manager of the Juventus football club since │                     │
│ are a helpful assistant.                    │ 2013. He was appointed as the manager of    │                     │
│ user                                        │ the club in 2013 after the departure of the │                     │
│ Who is the current manager of the Juventus  │ previous manager, Enzo32, who was sacked by │                     │
│ football club?                              │ the                                         │                     │
│ assistant                                   │                                             │                     │
│                                             │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ system                                      │  The Nazi Party was a fascist political     │ -3.508178472518921  │
│ You are Qwen, created by Alibaba Cloud. You │ party in Germany that was founded in 1923   │                     │
│ are a helpful assistant.                    │ by Adolf Hitler and other members of the    │                     │
│ user                                        │ Nazi Party. The Nazi Party was a fascist    │                     │
│ I think nazism did not exclude private      │ political party in Germany that was founded │                     │
│ ownership                                   │ in 1923 by Adolf Hitler and                 │                     │
│ assistant                                   │                                             │                     │
│                                             │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ system                                      │  To find the position of the top-left       │ 3.0420470237731934  │
│ You are Qwen, created by Alibaba Cloud. You │ corner of the face in the given image,      │                     │
│ are a helpful assistant.                    │ first find the position of the top-left     │                     │
│ user                                        │ corner of the face in the given image by    │                     │
│ Describe the steps involved in the facial   │ finding the intersection point of the two   │                     │
│ recognition process.                        │ lines that are perpendicular to the two     │                     │
│ assistant                                   │ edges of the given face                     │                     │
│                                             │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ system                                      │  Located in the southeastern part of the    │ 0.9301191568374634  │
│ You are Qwen, created by Alibaba Cloud. You │ country, Florida is the largest state in    │                     │
│ are a helpful assistant.                    │ the United States by area and the 34th      │                     │
│ user                                        │ largest by population. It is bordered by    │                     │
│ Where is Florida located in China?          │ the Atla

KeyboardInterrupt: 

In [23]:
import torch
tyt = tokenizer.decode(dataset["train"][0]["input_ids"])
inp = torch.tensor([dataset["train"][0]["input_ids"]]).to(reward_model.device)
res = reward_model(input_ids=inp)
res.logits

tensor([[3.0505]], device='cuda:0', grad_fn=<IndexBackward0>)